### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2022 ###

# Road Trip Optimizer #

#### Alex Gilmore (asgilmore@wisc.edu), Nandan Venkatesan (nvenkatesan2@wisc.edu), and Brendan Zimmer (btzimmer@wisc.edu)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-Model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-Discussion)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

add from google doc

## 2. Mathematical Model ##

We are utilizing an MCNF framework to solve our primary problem. The source and sink will be identical since we are interested in only round trips and we will be implementing a minimum trip length to prevent the model from cutting the trip short to reduce cost.

let p be a list of national parks and our starting location

let $x_{ij}$ be a matrix representing if an arc is chosen $\forall i \in p, \forall j \in p$

let $u_i$ be a vector representing the order of chosen nodes $\forall i \in p$

let $e_i$ be a vector of entrance fees $\forall i \in p$

let $l_i$ be a vector of est. lodging fees $\forall i \in p$

let $s_i$ be a vector of length of stays $\forall i \in p$

let $c_i$ be a vector of the total lodging and entrance fee cost $\forall i \in p$

let $g_{ij}$ be a matrix of gas prices $\forall i \in p, \forall j \in p$

let $D_{ij}$ be a matrix of  arc distances (miles) $\forall i \in p, \forall j \in p$

let $T_{ij}$ be a matrix of arc times (days) $\forall i \in p, \forall j \in p$

let M be the maximum days allowed to travel

let m be the minimum days allowed to travel

#### Miller-Tucker_Zemlin Model

\begin{align*}
\underset{x}{\max} \ & - \underset{(ij) \in p}\Sigma\ c_{i} x_{ij} - \underset{(ij) \in p}\Sigma\ g_{ij} x_{ij}  & \\
\text{s.t.} \ & \underset{(ij) \in p}\Sigma\ s_{i} x_{ij} + \underset{(ij) \in p}\Sigma\ T_{ij} x_{ij} \le M &\\
& - \underset{(ij) \in p}\Sigma\ s_{i} x_{ij} - \underset{(ij) \in p}\Sigma\ T_{ij} x_{ij} \le - m &\\
& \underset{i \in p} \Sigma\ x_{i, origin} \le 1 & \\
& - \underset{i \in p} \Sigma\ x_{i, origin} \le -1 & \\
& \underset{j \in p} \Sigma\ x_{origin, j} \le 1 & \\
& - \underset{j \in p} \Sigma\ x_{origin, j} \le -1 & \\
& \underset{j \in p} \Sigma\ x_{kj} \le 1, \forall k \in p & \\
& \underset{i \in p} \Sigma\ x_{ik} \le 1, \forall k \in p & \\
& u_i - u_j + n * x_{ij} \le n - 1, \forall i \in p, j \in p[2:end] &\\
& - \underset{k \in p} \Sigma\ x_{jk} + x_{ij} \le 0 , \forall i \in p, j \in p & \\
& & \\
& & \\
\end{align*}

Variables:

- $x_{ij}$ = $\left\{ \begin{array}{ll}
        1 \ \mbox{if arc $x_{ij}$ is in road trip path} \\
        0 \ \mbox{otherwise} & 
    \end{array} \right\}$
    
    
- $u_i$ = indicates the order of active nodes in the sequence

Constraints:
1. The total time traveled must be less than the maximum travel time.


2. The total time traveled must be more than the minimum desired travel time. This prevents the model from decreasing the trip length in an effort to reduce the trip cost. 


3, 4, 5, 6. This requires the origin to be included in the sequence loop


7, 8. This pair requires that each node has at most one entering and at most one leaving edge


9. This is the Miller-Tucker-Zemlin logical constraint. It ensures that u will increase along successive nodes in the sequence.


10. This is a logical constraint that ensures if a node has an active entering arc, then it must have an active leaving arc.

Objective Function:
- This is minimzing the collective cost of visiting parks + the gas cost of traveling between all parks. 

#### Adapative Subtour Elimination Model

## 3. Solution ##

### data entry

In [197]:
using Clp, Gurobi, JuMP, NamedArrays, CSV, DataFrames

In [346]:
df = DataFrame(CSV.File("park_data.csv"))

push!(df, ("source", "", 0, 0, 0, 3.5))

pTemp = df[!, "park name"]
p = Array{Symbol}(undef, length(pTemp))

# format parks list to symbols
for i in [1:1:length(pTemp);]
    pTemp[i] = replace(pTemp[i], " " => "_", "–" => "_", "." => "")
    p[i] = Symbol(pTemp[i])
end 

e = NamedArray(df[!, "entrance fee"], (p))

l = NamedArray(df[!, "lodging"], (p))

s = NamedArray(df[!, "est. length of stay"], (p))

parkGas = NamedArray(df[!, "gas price"], (p)) 

;

$c = s * (l + e)$

In [347]:
c = s .* (l + e)
;

In [348]:
dist = DataFrame(CSV.File("arc_data_dist_all.csv"))

time = DataFrame(CSV.File("arc_data_time_all.csv"))

size(time[1:49, 2:50])
size(p)

(49,)

In [351]:
D = Float64.(NamedArray(Matrix(dist[1:49,2:50]), (p, p), ("entering", "leaving")))
T = NamedArray(Matrix(time[1:49,2:50]), (p, p), ("entering", "leaving"))
;

In [352]:
T = T ./ 86400 # convert time from seconds to days
;

$g_{ij} =  \frac{parkGas_i + parkGas_j}{2} * \frac{1}{milesPerGallon} * D_{ij}$

In [353]:
# calculate the gas cost matrix

milesPerGallon = 20.0

g = copy(D)

for i in p
    for j in p
        g[i, j] = (parkGas[i] + parkGas[j]) / (2milesPerGallon) * g[i, j]
    end
end

println("acadia to yosemite distance: ", D[:Acadia, :Yosemite], " miles")
println("acadia to yosemite estimated gas cost: \$", g[:Acadia, :Yosemite])
println("acadia to yosemite time : ", T[:Acadia, :Yosemite], " days")

acadia to yosemite distance: 3100.0 miles
acadia to yosemite estimated gas cost: $795.15
acadia to yosemite time : 1.9795138888888888 days


In [356]:
# set travel length boundaries

maxTravel = 14
minTravel = 7
;

In [382]:
# set n (total number of parks)
n = length(p)
# n = 5
;

### model 

#### Miller-Tucker-Zemlin method

In [452]:
m = Model(Gurobi.Optimizer)

@variable(m, x[p, p], Bin)
@variable(m, 1 <= u[p] <= n)

@objective(m, Min, sum(c[i] * x[i, j] for i in p for j in p) 
    + sum(g[i, j] * x[i, j] for i in p for j in p))


# travel time constraints
@constraint(m, sum(s[i] * x[i, j] for i in p for j in p) 
    + sum(T[i, j] * x[i, j] for i in p for j in p) <= maxTravel)

@constraint(m, sum(s[i] * x[i, j] for i in p for j in p) 
    + sum(T[i, j] * x[i, j] for i in p for j in p) >= minTravel)

# no self loops constraint
@constraint(m, x_constr[i in p], x[i, i] == 0)

# no source --> sink loops constraints
# @constraint(m, sink_constr, x[:sink, :source] == 0)
# @constraint(m, source_constr, x[:source, :sink] == 0)

# start at source, end at sink constraints
@constraint(m, start_constr, sum(x[:source, j] for j in p) == 1)
@constraint(m, end_constr, sum(x[i, :source] for i in p) == 1)

# MTZ logical constraint
@constraint(m, MTZ[i in p, j in p[2:end]], u[i] - u[j] + n*x[i, j] <= n - 1)

# one out edge, one in edge constraints (at most)
@constraint(m, x_row_constr[k in p], sum(x[k, j] for j in p) <= 1)
@constraint(m, x_col_constr[k in p], sum(x[i, k] for i in p) <= 1)

# sequential connections logical constraint
@constraint(m, connections[i in p, j in p], sum(x[j, k] for k in p) + -1*x[i,j] >= -1 + 1)


# balance constraint
# @constraint(m, balance[k in p], sum(x[k, j] for j in p) - sum(x[i, k] for i in p) == b[k])

# if sum(s[i] * x[i, j] for i in p for j in p) 
#     + sum(T[i, j] * x[i, j] for i in p for j in p) >= minTravel
# @constraint(m, return_yet[i in p, j in p], sum(s[i] * x[i, j] for i in p for j in p) 
#                         + sum(T[i, j] * x[i, j] for i in p for j in p) + x[i,j] >= 7)
# then x[i, :sink] == 1
# @constraint(m, min[i in p], )

# if x[i, j] == 1, then x[j, i] == 0
# @constraint(m, one_way[i in p, j in p], x[i, j] <= x[j, i])

;

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-08


In [453]:
optimize!(m)
;

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4904 rows, 2450 columns and 136661 nonzeros
Model fingerprint: 0x077b2aed
Variable types: 49 continuous, 2401 integer (2401 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [4e+01, 4e+03]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 2583.9440000
Presolve removed 2720 rows and 227 columns
Presolve time: 0.27s
Presolved: 2184 rows, 2223 columns, 14835 nonzeros
Variable types: 47 continuous, 2176 integer (2176 binary)

Root relaxation: objective 5.975521e+02, 36 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  597.55214    0   13 2583.94400  597.55214  76.9%     -    0s
H    0     0 

In [467]:
xMTZ = value.(x);

In [462]:
MTZArcs = []
MTZCost = objective_value(m)

counter = 0

println("cost of road trip: \$", MTZCost, "\n")

for i in p
    for j in p
        if value(x[i, j]) > 0
            counter += 1
            println("arc ", counter, ": ", i, " --> ", j)
            append!(MTZArcs, [(String(i), String(j))])
        end
    end
end
 println("\n")
# for i in p
#     for j in p
#         if value(x[i, j]) > 0
#             counter += 1
#             println(x[i,j])
#         end
#     end
# end
MTZArcs

cost of road trip: $1810.1530000000002

arc 1: Acadia --> source
arc 2: Cuyahoga_Valley --> Acadia
arc 3: source --> Cuyahoga_Valley




3-element Vector{Any}:
 ("Acadia", "source")
 ("Cuyahoga_Valley", "Acadia")
 ("source", "Cuyahoga_Valley")

In [463]:
tripMileageMTZ = 0
tripDaysMTZ = 0

for i in p
    for j in p
        if value(x[i,j]) > 0
            tripMileageMTZ += D[i,j]
            tripDaysMTZ += T[i,j]
            tripDaysMTZ += s[i]
        end
    end
end

println("trip mileage: ", tripMileageMTZ)
println("trip length: ", tripDaysMTZ, " days")

trip mileage: 2799.0
trip length: 7.821122685185185 days


#### Adapative Subtour Elimination Method
This helps eliminate subtours by sort of formulating this similar to the TSP version but with only the set of nodes selected by the first optimal solution (that may include subtours). This limits subtour constraints by not forcing the model to visit all points

In [439]:
# Code from https://nbviewer.org/github/asmith28/cs524-su22/blob/main/TravelingSalesmanProblem.ipynb
# modified to suit our needs

# HELPER FUNCTION: returns the cycle containing the park START.
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in p
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x, specific_parks)
    nodesRemaining = specific_parks
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x, nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining, subtour)
    end
    return subtours
end
;



In [440]:
# parks = [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49]

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, x[p, p], Bin) # flow from one park to another

@constraint(m, c1[i in p], sum(x[i,:]) - sum(x[:,i]) == 0)
# @constraint(m, x_constr[i in p], x[i, i] == 0)

# enforce starting points
@constraint(m, sum(x[:source, :]) == 1)
@constraint(m, sum(x[:, :source]) == 1)

@constraint(m, self_loop[i in p], x[i, i] == 0) # no self-loops
# @constraint(m, sum(days[i]*sum(x[i,:]) for i in p) >= 7) # Min days spent
# @constraint(m, sum(days[i]*sum(x[i,:]) for i in p) <= 14) # Max days spent

# travel time constraints
@constraint(m, sum(s[i] * x[i, j] for i in p for j in p) 
    + sum(T[i, j] * x[i, j] for i in p for j in p) <= maxTravel)

@constraint(m, sum(s[i] * x[i, j] for i in p for j in p) 
    + sum(T[i, j] * x[i, j] for i in p for j in p) >= minTravel)


# @objective(m, Min, sum(cost[i, j] * x[i, j] for i in parks, j in parks))
@objective(m, Min, sum(c[i] * x[i, j] for i in p for j in p) 
    + sum(g[i, j] * x[i, j] for i in p for j in p))
;

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-08


The code below uses adaptive subtour elimination to add constraints that prevent subtours from forming. I've also modified the `getAllSubtours` helper function to only filter through the unique nodes provided by the first-run solution that may contain subtours. As Julia thinks this is optimal, we will add constraints to not have subtours for those. After this, Julia will iteratively run this code to determine the optimal solution with the no-subtour constraints applied

In [441]:
# Code from https://nbviewer.org/github/asmith28/cs524-su22/blob/main/TravelingSalesmanProblem.ipynb

sols = []
opt_sol = []

# We'll run the heuristic 30 times and hope we get an optimal solution
for iters = 1:30
    optimize!(m)
    # total  length of current tour
    println("Tour cost: ", objective_value(m))
    xx = value.(x) # save solution
    push!(sols, xx) # save solution

    nodes = []
    for i in p
        for j in p
            if Int(value(x[i, j])) == 1
                append!(nodes, [i, j])
            end
        end
    end

    subtours = getAllSubtours(xx, unique!(nodes))  # get all the subtours
    display(subtours)
    sleep(1)
    # get length of the subtour list
    len = length(subtours)
    if len == 1
        # solution is just a single tour!
        println("SOLVED!")

        # Calculate total days spent
        tot_days = 0
        for day in subtours[1]
            tot_days += s[day]
            println(subtours[1])
            println(subtours)
        end
        println("Total days spent: ", tot_days)

        # Show the optimal route
        for i in p
            for j in p
                if Int(value(x[i,j])) == 1
                    println([i,j])
                end
            end
        end
        opt_sol = subtours[1]
        println(opt_sol)

        break
    else
        for subtour in subtours
            L = length(subtour)
            # add constraints that cut off each subtour in the list (add two for each subtour)
            @constraint(m, sum(x[subtour[k+1], subtour[k]] for k = 1:L-1) <= L - 2)
            @constraint(m, sum(x[subtour[k], subtour[k+1]] for k = 1:L-1) <= L - 2)
        end
    end
end

Tour cost: 653.528


2-element Vector{Any}:
 [:Great_Smoky_Mountains, :New_River_Gorge, :Great_Smoky_Mountains]
 [:Indiana_Dunes, :source, :Indiana_Dunes]

Tour cost: 674.90775


1-element Vector{Any}:
 [:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]

SOLVED!
[:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]
Any[[:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]]
[:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]
Any[[:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]]
[:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]
Any[[:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]]
[:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]
Any[[:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]]
Total days spent: 8
[:Cuyahoga_Valley, :source]
[:Great_Smoky_Mountains, :Cuyahoga_Valley]
[:source, :Great_Smoky_Mountains]
[:Cuyahoga_Valley, :Great_Smoky_Mountains, :source, :Cuyahoga_Valley]


In [442]:
objective_value(m)

674.90775

In [429]:
println("Arcs in optimal solution are:")
path_opt = []
for i in p
    for j in p
        if Int(value(x[i,j])) == 1
            append!(path_opt, [[i,j]])
        end
    end
end

# This describes the __actual__ arcs of the solution rather than
# the nodes as shown when a subtour of length 1 is found
# NOTE: the way the solution is drawn will be the same as it's a closed loop
# but the optimal objective value depends on the arcs
path_opt

Arcs in optimal solution are:


3-element Vector{Any}:
 [:Cuyahoga_Valley, :source]
 [:Great_Smoky_Mountains, :Cuyahoga_Valley]
 [:source, :Great_Smoky_Mountains]

In [430]:
# println(c[:Acadia], " ", c[:Great_Smoky_Mountains])
# println(s[:Acadia], " ", s[:Great_Smoky_Mountains])
# println(l[:Acadia], " ", l[:Great_Smoky_Mountains])
# println(e[:Acadia], " ", e[:Great_Smoky_Mountains])


In [431]:
tripMileageASE = 0
tripDaysASE = 0

for i in p
    for j in p
        if value(x[i,j]) > 0
            tripMileageASE += D[i,j]
            tripDaysASE += T[i,j]
            tripDaysASE += s[i]
        end
    end
end

println("trip mileage: ", tripMileageASE)
println("trip length: ", tripDaysASE, " days")

trip mileage: 1739.0
trip length: 7.0984375 days


## 4. Results and Discussion ##

### MTZ Results

In [503]:
println("cost of road trip: \$", round(MTZCost, digits=2), "\n")
println("gas cost: \$", round(sum(convert(Array, g) .* Matrix(xMTZ))), 2)
println("lodging and entrance fee cost: \$", round(sum(convert(Array, c) .* Matrix(xMTZ)), digits=2), "0\n")
println("trip mileage: ", tripMileageMTZ)
println("trip length: ", round(tripDaysMTZ, digits=2), " days")

cost of road trip: $1810.15

gas cost: $572.02
lodging and entrance fee cost: $1238.00

trip mileage: 2799.0
trip length: 7.82 days


In [526]:
MTZArcs

3-element Vector{Any}:
 ("Acadia", "source")
 ("Cuyahoga_Valley", "Acadia")
 ("source", "Cuyahoga_Valley")

This version of our model found the optimal path to be UW-Madison --> Cuyahoga Valley --> Acadia --> UW-Madison.

### Adaptive Subtour Elimination Results

### General Discussion

## 5. Conclusion ##